# SCE-TTS: HiFi-GAN 학습

[SCE-TTS 프로젝트의 HiFi-GAN 학습용 문서](https://sce-tts.github.io/)

## 1. 할당된 GPU 확인

In [ ]:
import os
GPU_NAME = os.popen('nvidia-smi --query-gpu=name --format=csv,noheader').read().strip()
os.environ['GPU_NAME'] = GPU_NAME
print(f'GPU: {GPU_NAME}')

GPU: Tesla T4


## 2. 구글 드라이브 마운트
마운트할 구글 드라이브 내에 다음 파일이 존재하는지 꼭 확인하기
- `/Colab Notebooks/data/filelists.zip`

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 3. 필수 라이브러리 및 함수 불러오기

In [ ]:
import sys
from pathlib import Path

In [ ]:
%cd /content
!git clone --depth 1 https://github.com/sce-tts/TTS.git -b sce-tts
%cd /content/TTS
!python setup.py develop

## 4. 학습할 데이터셋 불러오기

학습에 사용할 음성 데이터를 구글 드라이브에서 가져옵니다.

In [ ]:
%cd /content/TTS
!cp "/content/drive/My Drive/Colab Notebooks/data/filelists.zip" ./filelists.zip
!rm -rf ./filelists
!unzip -q filelists.zip -d ./filelists

/content/TTS


## 5. 사전 학습 데이터 불러오기

사전 학습 데이터가 구글 드라이브에 존재하지 않을 경우,  
SleepingCE Speech Dataset의 사전 학습 데이터를 내려받기.

In [ ]:
%cd /content/TTS
!mkdir -p "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2"
if not Path("/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/config.json").exists():
    !gdown --id 1vRxp1RH-U7gSzWgyxnKY4h_7pB3tjPmU -O hifigan-v2.zip
    !unzip -q hifigan-v2.zip -d ./
    !cp -R ./hifigan-v2/* "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/"

/content/TTS


In [ ]:
%cd /content/TTS
if not Path("/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/scale_stats_new.npy").exists():
    !python TTS/bin/compute_statistics.py "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/config.json" "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/scale_stats_new.npy" --data_path "/content/TTS/filelists/wavs/"

/content/TTS


## 6. TensorBoard 실행

학습 진행 확인용

In [ ]:
%load_ext tensorboard
%tensorboard --logdir="/content/drive/My Drive/Colab Notebooks/data/hifigan-v2"

## 7. HiFi-GAN 학습 진행

In [ ]:
%cd /content/TTS
# !(python TTS/bin/train_vocoder_gan.py \
#     --continue_path "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/CONTINUE_DIRECTORY")
!(python TTS/bin/train_vocoder_gan.py \
    --config_path "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/config.json" \
    --coqpit.data_path "/content/TTS/filelists/wavs"  \
    --coqpit.audio.stats_path "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/scale_stats_new.npy"  \
    --coqpit.output_path "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/"  \
    --coqpit.num_loader_workers 2  \
    --coqpit.num_val_loader_workers 2  \
    --restore_path "/content/drive/My Drive/Colab Notebooks/data/hifigan-v2/model_file.pth.tar")